<a href="https://colab.research.google.com/github/comradyo/kis/blob/main/lr-8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа 8. Анализ одноканальной системы массового обслуживания с учетом приоритета заявок

## Задание 1.

Система электронного документооборота с автоматическим классификатором поступающей корреспонденции должна учитывать различную важность документов, количество типов документов $P$, средняя интенсивность поступления заявок $\lambda_1, \lambda_2, \ldots, \lambda_P$, средняя интенсивность обслуживания $\mu_1, \mu_2, \ldots, \mu_P$ соответственно, очередь неограничена. Приоритеты заявок $p_1<p_2<\ldots<p_P$.

Оценить среднее время нахождения каждого типа заявки в очереди (теоретически и экспериментально) для одного из 4 случаев:
- относительный фиксированный приоритет (V1);
- абсолютный фиксированный приоритет (V2);
- приоритет с зависимой задержкой без прерывания (V3);
- приоритет с зависимой задержкой с прерываниями (V4);

в соответствии с вариантом.

In [94]:
Variant<-8
set.seed(Variant) 
P<-sample(c(4, 6),1)
V<-sample(c("V1", "V2", "V3", "V4"),1)
if ((V=="V3") | (V=="V4")){
    bs<-sort(sample(c(1:10),P))
}
lambdas<-runif(P)
mus<-runif(P,1,3)
View(data.frame(P,V))
if ((V=="V3") | (V=="V4")) {View(data.frame(lambdas, mus, bs))}
if ((V=="V1") | (V=="V2")) {View(data.frame(lambdas, mus))}


P,V
<dbl>,<chr>
6,V4


lambdas,mus,bs
<dbl>,<dbl>,<int>
0.089301011,1.528918,1
0.432391373,1.553064,2
0.544962116,2.042214,4
0.138224346,1.447378,7
0.927812252,1.815850,9
0.001301721,2.229751,10


Теоретическая часть:



$$\lambda=\sum_{p=1}^P\lambda_p$$
$$\frac{1}{\mu}=\sum_{p=1}^P \frac{\lambda_p}{\lambda}\cdot \frac{1}{\mu_p}$$
$$\rho_p=\frac{\lambda_p}{\mu_p}$$
$$\rho=\frac{\lambda}{\mu}=\sum_{p=1}^P \rho_p$$
$$W_0=\sum_{p=1}^P\frac{\rho_p}{\mu_p}$$

Рекуррентная зависимость $W_p$ от $W_i$ для заявок с низшими приоритетами **для системы приоритета с зависимой задержкой с прерыванием**:
$$0\leq\rho<1: W_p=\frac{1}{1-\sum_{i=p+1}^P\rho_i\cdot\left(1-\frac{b_p}{b_i}\right)}\cdot$$ $$\left(\frac{W_0}{1-\rho}+\sum_{i=p+1}^P\frac{\rho_i}{\mu_p}\cdot \left(1-\frac{b_p}{b_i}\right)-\sum_{i=1}^{p-1}\frac{\rho_i}{\mu_i}\cdot\left(1-\frac{b_i}{b_p}\right)-\sum_{i=1}^{p-1}\rho_i\cdot W_i\cdot \left(1-\frac{b_i}{b_p}\right)\right)$$

Для более наглядного представления введем обозначения:

$$A = 1 - \sum_{i = p+1}^P \rho_i \cdot \left( 1 - \frac{b_p}{b_i} \right) $$

$$B = \frac{W_0}{1-\rho} $$

$$C = \sum_{i=p+1}^P\frac{\rho_i}{\mu_p}\cdot \left(1-\frac{b_p}{b_i}\right) $$

$$D = \sum_{i=1}^{p-1}\frac{\rho_i}{\mu_i}\cdot\left(1-\frac{b_i}{b_p}\right) $$

$$E = \sum_{i=1}^{p-1}\rho_i\cdot W_i\cdot \left(1-\frac{b_i}{b_p}\right) $$

Тогда формулу для нахождения $W_p$ можно записать в виде:

$$ W_p = \frac{B + C - D - E}{A}$$

Реализуем теоретическую часть в среде R:

In [95]:
# функция суммирования
summ <- function(from, to, expr) {
  if (from > to) {
    return(0)
  }
  res <- 0
  for (i in (from:to)) {
    res <- res + expr(i)
  }
  return(res)
}

In [96]:
rhos <- lambdas/mus
rho <- sum(rhos)
rho

# rho > 1, уменьшим увеличим mus на 0.5
mus <- mus + 0.5

rhos <- lambdas/mus
rho <- sum(rhos)
rho

[1] 1.210704

[1] 0.9410795

Рекуррентная зависимость $W_p$ от $W_i$ для заявок с низшими приоритетами **для системы приоритета с зависимой задержкой с прерыванием**:
$$0\leq\rho<1: W_p=\frac{1}{1-\sum_{i=p+1}^P\rho_i\cdot\left(1-\frac{b_p}{b_i}\right)}\cdot$$ $$\left(\frac{W_0}{1-\rho}+\sum_{i=p+1}^P\frac{\rho_i}{\mu_p}\cdot \left(1-\frac{b_p}{b_i}\right)-\sum_{i=1}^{p-1}\frac{\rho_i}{\mu_i}\cdot\left(1-\frac{b_i}{b_p}\right)-\sum_{i=1}^{p-1}\rho_i\cdot W_i\cdot \left(1-\frac{b_i}{b_p}\right)\right)$$

Для более наглядного представления введем обозначения:

$$A = 1 - \sum_{i = p+1}^P \rho_i \cdot \left( 1 - \frac{b_p}{b_i} \right) $$

$$B = \frac{W_0}{1-\rho} $$

$$C = \sum_{i=p+1}^P\frac{\rho_i}{\mu_p}\cdot \left(1-\frac{b_p}{b_i}\right) $$

$$D = \sum_{i=1}^{p-1}\frac{\rho_i}{\mu_i}\cdot\left(1-\frac{b_i}{b_p}\right) $$

$$E = \sum_{i=1}^{p-1}\rho_i\cdot W_i\cdot \left(1-\frac{b_i}{b_p}\right) $$

Тогда формулу для нахождения $W_p$ можно записать в виде:

$$ W_p = \frac{B + C - D - E}{A}$$

In [97]:
W0 <- sum(rhos/mus)

A <- function(p) {
  return(
    1 - summ(p + 1, P, function(i) {
      return(
        rhos[i] * (1 - bs[p] / bs[i])
        )
      }
    )
  )
}

B <- W0 / (1 - rho)

C <- function(p) {
  return(
    summ(p+1, P, function(i) {
      return(
        (rhos[i] / mus[p]) * (1 - bs[p] / bs[i])
      )
    })
  )
}

D <- function(p) {
  return(
    summ(1, p-1, function(i) {
      return(
        (rhos[i] / mus[i]) * (1 - bs[i] / bs[p])
        )
    })
  )
}

E <- function(p, W) {
  return(
    summ(1, p-1, function(i) {
      return(
        rhos[i] * W[i] * (1 - bs[i] / bs[p])
        )
    })
  )
}

In [98]:
Ws <- double(P)
Ws[1] <- (B + C(1) - D(1)) / A(1)

for (p in (2:P)) {
  Ws[p] <- (B + C(p) - D(p) - E(p, Ws)) / A(1)
}

printf <- function(...)print(sprintf(...))

for (i in 1:P) {
   cat(i,"\t",  Ws[i],'\n')
}

1 	 23.4885 
2 	 21.47999 
3 	 12.93006 
4 	 5.40369 
5 	 2.784587 
6 	 1.511347 


Практическая часть, реализованная в среде R:

In [115]:
N <- 10000000  # Число тиков (прибавления дельт времени)
dt <- 0.001      # Тик (Дельта времени)
curTime <- 0   # Текущее время симуляции

buffer <- c() # буфер заявок в систему, обрабатывается первая заявка
bufferReceiveAt <- c() # буфер моментов времени, когда заявки появились в системе
bufferPriority <- c() # буфер приоритетов заявок
bufferType <- c() # буфер типов заявок

timeInSystByType <- double(P) # сколько времени находились в системе заявки каждого из P типов
numOfSuccesfulReqsByType <- integer(P) # суммарное число обработанных заявок каждого из P типов

receiveReqAtByPriority <- double(P)
for (p in 1:P) {
  receiveReqAtByPriority[p] <- rexp(1, lambdas[p])
}

getHandleDuration <- function(p) {
  return (rexp(1, mus[p]))
}

for (i in 1:N) {
  curTime <- curTime + dt

  # Поступление новых заявок
  for (p in 1:P) {
    if (curTime >= receiveReqAtByPriority[p]) {
      buffer <- append(buffer, getHandleDuration(p))
      bufferReceiveAt <- append(bufferReceiveAt, receiveReqAtByPriority[p])
      bufferType <- append(bufferType, p)

      receiveReqAtByPriority[p] <- receiveReqAtByPriority[p] + rexp(1, lambdas[p])
    }
  }

  # обновление приоритетов
  for (j in 1:length(buffer)) {
    bufferPriority[j] <- (curTime - bufferReceiveAt[j]) * bs[bufferType[j]]
  }

  # сокращение времени обработки текущей программы, которая находится на сервере.
  if (length(buffer) > 0) {
    maxPriorityReqID <- which.max(bufferPriority)

    # swap
    if (maxPriorityReqID != 1) {
      #print(maxPriorityReqID)
      b1 <- buffer[1]
      b1RecAt <- bufferReceiveAt[1]
      b1Priority <- bufferPriority[1]
      b1Type <- bufferType[1]

      buffer[1] <- buffer[maxPriorityReqID]
      bufferReceiveAt[1] <- bufferReceiveAt[maxPriorityReqID]
      bufferPriority[1] <- bufferPriority[maxPriorityReqID]
      bufferType[1] <- bufferType[maxPriorityReqID]

      buffer[maxPriorityReqID] <- b1
      bufferReceiveAt[maxPriorityReqID] <- b1RecAt
      bufferPriority[maxPriorityReqID] <- b1Priority
      bufferType[maxPriorityReqID] <- b1Type
    }

    buffer[1] <- buffer[1] - dt
    if (buffer[1] <= 0) {
      numOfSuccesfulReqsByType[bufferType[1]] <- numOfSuccesfulReqsByType[bufferType[1]] + 1
      timeInSystByType[bufferType[1]] <- timeInSystByType[bufferType[1]] + curTime - bufferReceiveAt[1]

      buffer <- buffer[-1]
      bufferReceiveAt <- bufferReceiveAt[-1]
      bufferPriority <- bufferPriority[-1]
      bufferType <- bufferType[-1]
    }
  }
}

In [116]:
numOfSuccesfulReqsByType
lambdas
timeInSystByType

[1]  852 4335 5500 1384 9250   17

[1] 0.089301011 0.432391373 0.544962116 0.138224346 0.927812252 0.001301721

[1] 19441.43303 54501.27073 37103.79417  5603.63759 30396.90298    53.44555

In [117]:
res <- timeInSystByType / numOfSuccesfulReqsByType
for (p in 1:P) {
  print(res[p])
}

[1] 22.81858
[1] 12.57238
[1] 6.746144
[1] 4.048871
[1] 3.286152
[1] 3.143856
